In [74]:
from PyBlockchain import Miner, Node, Block, Blockchain

## step 0: initialize chain

we initialize the blockchain ("ledger") with the genesis block. in our toy example, our ledger will be a python list containing Block objects.

In [75]:
genesis_block = Block('0')
genesis_block.show()

>>> block hash: 0
>>> block index: 0
>>> previous block hash: 0
>>> root hash: 0
>>> nonce:  0


In [76]:
ledger = Blockchain(genesis_block)

## step 1: creating a new block
nodes create a new Block for every new batch of transactions.

In [77]:
Node_Alice = Node(ledger)
Node_Bob = Node(ledger)

In [78]:
block1 = Node_Alice.generate_new_block('S2M')
block1.show()

>>> block hash: 0
>>> block index: 1
>>> previous block hash: 0
>>> root hash: S2M
>>> nonce:  0


## step 2: attempt to add new block to the ledger

a new Block cannot be added to the blockchain until its hash demonstrates **proof of work**. 

in our toy example, an acceptable hash is one where h(previous hash + transaction) starts with the character "c". h(x) in this case is sha1 hash function.

for example, sha1("hello World11") is 'cc8e9d1fbe83d1ce4b1bf546ebff3b735e4ff500'. since this hash output starts with the character "c", it will be considered to be a valid **proof of work** and thus its associated Block can be added to the ledger.

in practice, an acceptable hash, usually sha256, must start with three zeros (eg "000abfds..."). 

In [79]:
# can new block be added to blockchain?
Node_Alice.update_blockchain(block1)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... False
>>> update failed


False

## step 3: miners demonstrate proof of work
through brute force, miners find the correct hash that satisfies conditions. once the correct hash is identified, it will be assigned to that block, thereby enabling addition to the ledger.

In [80]:
Miner_Adam = Miner()

In [81]:
good_hash = Miner_Adam.calculate_hash_block(block1)
print "valid hash:", good_hash

valid hash: 0c56c07f2681d154b79023e388b4c3e01eb49cd1


In [82]:
# miner updates the block with the hash
Miner_Adam.update_block_hash(block1, good_hash)

In [83]:
block1.show()

>>> block hash: 0c56c07f2681d154b79023e388b4c3e01eb49cd1
>>> block index: 1
>>> previous block hash: 0
>>> root hash: S2M
>>> nonce:  18


In [84]:
# with the new hash, verify new block can be added to blockchain
Node_Alice.update_blockchain(block1)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True
>>> successfully updated blockchain...


True

In [85]:
block1.show()

>>> block hash: 0c56c07f2681d154b79023e388b4c3e01eb49cd1
>>> block index: 1
>>> previous block hash: 0
>>> root hash: S2M
>>> nonce:  18


### simulate another transaction

In [86]:
block2 = Node_Alice.generate_new_block('M1J')
block2.show()

>>> block hash: 0
>>> block index: 2
>>> previous block hash: 0c56c07f2681d154b79023e388b4c3e01eb49cd1
>>> root hash: M1J
>>> nonce:  0


In [87]:
Node_Alice.check_block(block2)  # verify new block can be added to blockchain

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... False


False

In [88]:
good_hash = Miner_Adam.calculate_hash_block(block2)
print "valid hash:", good_hash

valid hash: 01c89bb59979db4ebc6fd5dd14ec4fb942114353


In [89]:
# since we've found the seal, we can update the block, thus approving it for blockchain addition
Miner_Adam.update_block_hash(block2, good_hash)

In [90]:
Node_Alice.check_block(block2)  # verify new block can be added to blockchain

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True


True

In [91]:
block2.show()

>>> block hash: 01c89bb59979db4ebc6fd5dd14ec4fb942114353
>>> block index: 2
>>> previous block hash: 0c56c07f2681d154b79023e388b4c3e01eb49cd1
>>> root hash: M1J
>>> nonce:  13


In [92]:
# with the new hash, verify new block can be added to blockchain
Node_Alice.update_blockchain(block2)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True
>>> successfully updated blockchain...


True

In [93]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print blocks.fetch_transaction()

transaction 0 : 0
transaction 1 : S2M
transaction 2 : M1J


In [94]:
# see if record 2 is valid
print "hash signatures match?", 
Node_Alice.blockchain_copy.blockchain[1].block_hash == Miner.calculate_hash_block(Node_Alice.blockchain_copy.blockchain[1])

hash signatures match?

True

# what is record 1 was altered?

In [95]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print blocks.fetch_transaction()

transaction 0 : 0
transaction 1 : S2M
transaction 2 : M1J


In [96]:
Node_Alice.blockchain_copy.blockchain[1].data = "cheater"

In [97]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print blocks.fetch_transaction()

transaction 0 : 0
transaction 1 : dummy data
transaction 2 : M1J


In [98]:
# see if record 2 is valid
print "hash signatures match?", 
Node_Alice.blockchain_copy.blockchain[1].block_hash == Miner.calculate_hash_block(Node_Alice.blockchain_copy.blockchain[1])

hash signatures match?

False